In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
eco = pd.read_excel("../data/TEIS-NSS Project Data 10-2022.xlsx",sheet_name=('ECO with Exit21-22'))
elig = pd.read_excel("../data/TEIS-NSS Project Data 10-2022.xlsx",sheet_name=('Elig Timeline Rpt 2018-2022'))

In [ ]:
eco['CHILD_ID']

In [ ]:
elig['Child ID']

In [ ]:
eco.head(10)

In [ ]:
eco.tail(10)

In [ ]:
eco.info()

In [ ]:
eco['<Calc> Months in Program'].head()

In [ ]:
elig.info()

In [ ]:
elig.head()

In [ ]:
elig.tail()

### Clean up eco table.
With Child ID, District, and day count.
Rename columns to child_id, district, day_count

In [ ]:
eco_count = eco[["CHILD_ID", "DISTRICT", "<Calc> Months in Program","Ent SOCIAL_SCALE","Ent KNOWLEDGE_SCALE","Ent APPROPRIATE_ACTION_SCALE","Exit SOCIAL_SCALE","Exit KNOWLEDGE_SCALE","Exit APPROPRIATE_ACTION_SCALE", 'OC1 Data Check']]

In [ ]:
eco_count.columns = ['child_id', 'district', 'month_count', 'ent_social', 'ent_knowledge', 'ent_app_action', 'exit_social', 'exit_knowledge', 'exit_app_action', 'oc1']
eco_count.info()

In [ ]:
eco_count['district'].head(20)

### Clean up ELIG table.
With Child ID, Elig Category, Day Count
Rename columns to child_id, elig_cat
Change data to float

In [ ]:
elig_cat1 = elig[["Child ID", "Init. Elig. Category", 'District']]

In [ ]:
elig_cat1.columns = ['child_id', 'elig_cat', 'district']

In [ ]:
elig_cat1.info()

In [ ]:
elig_cat1.head(20)

In [ ]:
elig_cat1.groupby(['elig_cat'])['elig_cat'].count()

In [ ]:
elig_cat1["child_id"] = pd.to_numeric(elig_cat1["child_id"], downcast="float")

In [ ]:
elig_cat1.info()

### Merge the two new tables.

In [ ]:
eco_elig = pd.merge(eco_count, elig_cat1, how='outer',on=['child_id', 'district'])

In [ ]:
eco_elig.info()

In [ ]:
eco_elig.head(20)

In [ ]:
eco_elig.info()

In [ ]:
eco_elig = eco_elig[eco_elig['oc1'].notna()]


In [ ]:
eco_elig

### Add column for calculation of difference of scores

In [ ]:
eco_elig['social_change'] = eco_elig['exit_social'] - eco_elig['ent_social']
eco_elig['knowledge_change'] = eco_elig['exit_knowledge'] - eco_elig['ent_knowledge']
eco_elig['app_action_change'] = eco_elig['exit_app_action'] - eco_elig['ent_app_action'] 

eco_elig.head(20)

In [ ]:
print(eco_elig[['ent_social', 'exit_social', 'social_change']].head(20))

In [ ]:
eco_elig.info()

### Subset with just changes and child_id
Start graphs on this data

In [ ]:
eco_change_cat = eco_elig[['social_change', 'knowledge_change', 'app_action_change','elig_cat','district']]

In [ ]:
eco_change_cat.value_counts('elig_cat').count

In [ ]:
eco_change_cat

In [ ]:
sum_change = eco_change_cat.groupby(['elig_cat']).sum()


In [ ]:
sum_change

In [ ]:
sum_change.plot.bar(stacked = True)

### Try again with the nulls dropped

In [ ]:
drop_change_by_cat = eco_change_cat.dropna()

In [ ]:
drop_change_by_cat.head(30)

In [ ]:
drop_change_by_cat.info()

In [ ]:
drop_change_by_cat.value_counts('elig_cat').count

In [ ]:
display(drop_change_by_cat)

In [ ]:
drop_change_by_cat.groupby(['elig_cat']).sum()

In [ ]:
sns.barplot(data = drop_change_by_cat, x = 'social_change')

In [ ]:
dcbc = drop_change_by_cat.groupby(['elig_cat'], as_index = False).mean()

In [ ]:
dcbc.info()

In [ ]:
display(dcbc)

### Social Change graph

In [ ]:
sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(7, 6))

sns.boxplot(x="social_change", y="elig_cat", data=drop_change_by_cat,
            whis=[0, 100], width=.6, palette="vlag")

sns.stripplot(x="social_change", y="elig_cat", data=drop_change_by_cat,
              size=4, color=".3", linewidth=0)

ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

### Knowledge Change Graph

In [ ]:
sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(7, 6))

sns.boxplot(x="knowledge_change", y="elig_cat", data=drop_change_by_cat,
            whis=[0, 100], width=.6, palette="vlag")

sns.stripplot(x="knowledge_change", y="elig_cat", data=drop_change_by_cat,
              size=4, color=".3", linewidth=0)


ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

### Appropriate Action Change

In [ ]:
sns.set_theme(style="ticks")

f, ax = plt.subplots(figsize=(7, 6))

sns.boxplot(x="app_action_change", y="elig_cat", data=drop_change_by_cat,
            whis=[0, 100], width=.6, palette="vlag")

sns.stripplot(x="app_action_change", y="elig_cat", data=drop_change_by_cat,
              size=4, color=".3", linewidth=0)

ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

### Each child is associated with a Point of Entry (POE) office, as indicated in column A of the "ECO with Exit21-22" column. Do the above comparison by POE as well, similar to the calculations in the "ECO by POE" tab.

In [ ]:
drop_change_by_cat.info()

In [ ]:
poe1 = drop_change_by_cat[['social_change','knowledge_change', 'app_action_change', 'elig_cat']]

In [ ]:
poe = drop_change_by_cat.dropna()

In [ ]:
poe = poe[['social_change','knowledge_change', 'app_action_change', 'district']]

In [ ]:
poe_sum = poe.groupby(['district'], as_index = False).sum()

In [ ]:
poe_sum.plot(kind = 'bar',figsize = (10,6))

In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")

#sns.color_palette("dark:#5A9_r", as_cmap=True)

f, ax = plt.subplots(figsize=(7, 6))

sns.boxplot(x="app_action_change", y="district", data=poe,
            whis=[0, 100], width=.6, palette="vlag")

sns.stripplot(x="app_action_change", y="district", data=poe,
              size=4, color=".3", linewidth=0)

ax.xaxis.grid(True)
ax.set(ylabel="")
sns.despine(trim=True, left=True)

In [ ]:
sns.barplot(data=poe, x='district', y='app_action_change',\
            order= ('FT','UC','SW','ET','NW','SE','GN','SC','MD'))

In [ ]:
sns.barplot(data=poe1, x='elig_cat', y='app_action_change')

In [ ]:
sns.barplot(data=poe1, x='elig_cat', y='social_change')

In [ ]:
poe1.head()

In [ ]:
elig1avg = poe1[['social_change', 'app_action_change', 'knowledge_change']].sum()
elig1avg.plot(kind="bar", title='Developmental evaluation (delay)')


In [ ]:
elig1avg = poe1[['social_change', 'app_action_change', 'knowledge_change']].mean()
elig1avg.plot(kind="bar", title='Developmental evaluation (delay)')